In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
from DTA.enrichment_plotters import polar_plot, get_file_list, get_helices, read_rep
from DTA.polarDensity_helper import Coord_Get, get_header_info
from DTA.site_distributions import outline_site, make_symmetric_sites, combine_sites, plot_density, Site
plt.rcParams['axes.grid'] = False 

In [ ]:
# Enrichment plot colormap
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr', 256)
newcolors = np.concatenate([depleted(np.linspace(0.35, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

# Identify one or more sites from enrichment plots

## Where are the files?

In [ ]:
lipids = ["POPG", "POPE"]
leaflets = ['low', 'upp']
root = Path("/Users/ezry/Projects/ELIC_PE_PG/aggregated/")
replicas = ["rep1", "rep3", "liam"]
helix_definitions = root.joinpath(replicas[2]) #where are the coordinates for the transmembrane helices?

## Boilerplate read and parse the files

In [ ]:
enrichments = None
counts = None
helices_upr, helices_lwr = get_helices(helix_definitions)
for rep in replicas:
    reppath = root.joinpath(rep)
    file_list = get_file_list(reppath, lipids)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(file_list[0])
    cts, rich = read_rep(file_list, lipids, leaflets, enrich=True)
    if counts is None:
        counts = cts
        enrichments = rich
    else:
        counts = counts+cts
        enrichments = enrichments+rich
    

counts = counts/len(replicas)
enrichments = enrichments/len(replicas)
thetas = np.unique(theta.flatten())

generic_settings ={'Ntheta':Ntheta,
                   'dr':dr, 
                   'dth':dth,
                   'exrho':1,
                   'frames':frames,
                   }



## Define some putative sites

In [ ]:
site1def = {'theta_start':5, 
            'width':3, 
            'inner_r':24, 
            'outer_r':30, 
            'sitename':"Site 1 ",
            }

site2def = {'theta_start':9,
            'width':4,
            'inner_r':22,
            'outer_r':32,
            'sitename':"Site 2 ",
            }

dummy_sites_1 = make_symmetric_sites([], **site1def, **generic_settings)
dummy_sites_2 = make_symmetric_sites([], **site2def, **generic_settings)

## Plot enrichments (you may need to go back and update your site definitions)

In [ ]:
# Remove pore density:
enrichments.at['POPE', 'low'][0:3,:]=0

fig, axes = polar_plot(enrichments, 
                       theta, 
                       radius, 
                       lipids, 
                       helices_lwr, 
                       helices_upr, 
                       colorbychain=False, 
                       vmin=0.75, 
                       vmax=1.5, 
                       vmid=1,
                       figheight=8,
                       figwidth=8,
                       cmap=my_cmap)

#this is where you decide which leaflet each site refers to 0==outer leaflet, 1==inner leaflet
for site in dummy_sites_1:
    axes[0] = outline_site(axes[0], site)

for site in dummy_sites_2:
     axes[1] = outline_site(axes[1], site)


fig.tight_layout()
plt.savefig(root.joinpath("ELIC_enrichments.pdf"))
plt.show()

# Get Accessible Area
This process should be done for each site.
1. Read in the data (update the user parameters according to your paths and leaflet of interest)
2. Get the total area of the site. This is your initial guess.
3. Get/read the bulk bead distribution
4. Calculate the accessible area and compare to your guess
5. If the accessible area remains very similar to your guess, you're done. Otherwise, try a guess that's closer to the estimated accessible area and go back to 3.

## Step 1: Read data

In [ ]:
# User parameters
lipid = "DPPC"
leaf = "upp"
the_site_def = site1def
data_root = Path("/Users/ezry/Projects/ELIC_DPPC/elicPE17_DPPC/")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

# Update exrho and frames based on the above since it will differ for DPPC. All other parameters are constant across systems.
the_site_def['exrho'] = exrho
the_site_def['frames'] = frames


the_sites = make_symmetric_sites(the_data, **the_site_def)
site1_symmetric = combine_sites(the_sites, exrho, f"{the_site_def['sitename'][:-1]}, Symmetric", symmetric=True)

sites = {'a_site':the_sites[0], 
         'the_site_symmetric':site1_symmetric,
         }

## Step 2: get the total area of the site in question

In [ ]:
print(f"{'Site Name':<20}:{'Atotal':>8}|{'peak'}")
for name, site in sites.items():
    print(f"{site.title:<20}:{np.round(site.area,1):>8}|{np.round(site.peak,0)}")

## Step 3: get the bulk density 
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl

OR 

using the get_counts.ipynb notebook (requires MDAnalysis):
1. Optional: set make_movies to True. This will export mp4s of the bead counts as a top-down heatmap; may be useful for debugging.
2. Update the path to the trajectory and load it
3. Assign leaflets (either using a resid cutoff or a more robust method)
4. update ``areas`` to be a list of one or more areas of interest

Either method will output data files of the site bead counts over time.

Provide the path to the outputs here:

In [ ]:
area = 49
bulk_counts_path = Path(f"../accessible_area/counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

## Examine the site densities and compare with the bulk mode

In [ ]:
rows = 1
cols = len(sites)
fig, axes = plt.subplots(rows,cols, figsize=(10*cols,7*rows))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)

fig.tight_layout()
plt.savefig(data_root.joinpath("Raw_distributions.pdf"))

## Step 4: The accessible area is...
ASSUMING $\left(A_\mathrm{site} \approx A_\mathrm{bulk}\right) \forall A_\mathrm{site}$

$A_\mathrm{acc} = A_\mathrm{total} \frac{M_\mathrm{site}}{M_\mathrm{bulk}}$

In [ ]:
print(f"{'Site Name':<20}:{'Accessible Area':>5}")
areas = []
warnings = []
for name, site in sites.items():
    counts = site.counts.astype(int)
    bincounts = np.bincount(counts.flatten())
    the_mode = np.argmax(bincounts)
    if the_mode == 0:
        warnings.append(f"Warning: found an experimental mode of 0 for '{site.title},' using second highest peak")
        the_mode = np.argmax(bincounts[1:])+1
    A_acc = site.area * the_mode / bulk_mode
    areas.append(A_acc)
    print(f"{site.title:<20}:{np.round(A_acc,1):>5}")

for warn in warnings:
    print(warn)

In [ ]:
print(f"The average site area is: {np.mean(areas[1:])} square angstroms")
